# 1 Making the reference catalogues


We need a photometric and astrometric reference. We are going to take the HSC PanSTARRS reference as a base and cross match in the VIDEO JHKs fluxes from HELP. 


The final astrometric reference catalogue will probably be GAIA DR2 astrometry and PanSTARSS plus either 2MASS or the original VISTA catalogues.

In [28]:
from lsst.meas.algorithms.htmIndexer import HtmIndexer
from lsst.geom import SpherePoint 
from lsst.geom import degrees

import numpy as np
from astropy.io import fits



In [29]:
ORIG = ''
EX_CAT = "../dmu0/dmu0_PanStarrs/data/ps1_pv3_3pi_20170110_GmagLT19/133200.fits"
EX_MS = "../dmu0/dmu0_PanStarrs/data/ps1_pv3_3pi_20170110_GmagLT19/master_schema.fits"

In [30]:
def getShards(ra, dec, radius):
    htm = HtmIndexer(depth=7)
    shards, onBoundary = htm.getShardIds(SpherePoint(ra*degrees, dec*degrees), radius*degrees)
    return shards
getShards(35.428,  -4.90777, 1.0)

array([133201, 133202, 133203, 133314, 133200, 133206, 133209, 133213,
       133214, 133215, 133312, 133313, 133315, 133321, 133325, 133326,
       133327, 134048, 134049, 134051, 134054, 134062])

In [2]:
#Everything in SXDS DUD region
ps_refcats = getShards(36.,  -5.0, 2.0)
files = ''
for c in ps_refcats:
    files += '{}.fits,'.format(c)
print('scp ir-shir1@login.hpc.cam.ac.uk:~/rds/rds-iris-ip005/data/public/PanSTARRS/ps1_pv3_3pi_20170110_GmagLT19/\{'
      +files+'\} ./')

scp ir-shir1@login.hpc.cam.ac.uk:~/rds/rds-iris-ip005/data/public/PanSTARRS/ps1_pv3_3pi_20170110_GmagLT19/\{133200.fits,133201.fits,133202.fits,133203.fits,133206.fits,133209.fits,133212.fits,133213.fits,133214.fits,133215.fits,133282.fits,133289.fits,133293.fits,133312.fits,133313.fits,133314.fits,133315.fits,133317.fits,133318.fits,133319.fits,133321.fits,133322.fits,133323.fits,133324.fits,133325.fits,133326.fits,133327.fits,134049.fits,133204.fits,133205.fits,133207.fits,133208.fits,133210.fits,133211.fits,133234.fits,133241.fits,133245.fits,133280.fits,133281.fits,133283.fits,133288.fits,133290.fits,133291.fits,133292.fits,133294.fits,133295.fits,133316.fits,133320.fits,133364.fits,133365.fits,133367.fits,133368.fits,133370.fits,133371.fits,133372.fits,133968.fits,134048.fits,134050.fits,134051.fits,134052.fits,134054.fits,134055.fits,134061.fits,134062.fits,134063.fits,134080.fits,\} ./


In [3]:
sql = """
SELECT 
    ra, 
    dec, 
    f_vista_j, 
    ferr_vista_j, 
    f_vista_h, 
    ferr_vista_h, 
    f_vista_ks, 
    ferr_vista_ks 
FROM herschelhelp.main 
WHERE 
    ra > 33.5 
    AND ra < 38. 
    AND dec > -6.5 
    AND dec < -3. 
    AND f_vista_ks IS NOT NULL 
    AND m_gpc1_g < 19.
"""

In [4]:
import pyvo as vo
service = vo.dal.TAPService(
    "https://herschel-vos.phys.sussex.ac.uk/__system__/tap/run/tap"
)

In [5]:
import time
job = service.submit_job(sql)
start_time = time.time()
job.run()
job_url = job.url
job.wait(phases='COMPLETED')
print('Job {} running after {} seconds.'.format(job.phase, round(time.time() - start_time)))

Job COMPLETED running after 42 seconds.


In [6]:
result = job.fetch_result()
v_flux = result.table

In [7]:
len(v_flux)

11075

In [8]:
def clean_table(table):
    """Take a table produced by a VO query and remove all empty columns
    
    Often many columns are empty and make the tables hard to read.
    The function also converts columsn that are objects to strings.
    Object columns prevent writing to fits.
    
    Inputs
    =======
    table,    Astropy.table.Table
        The input table
    
    Returns
    =======
    table,    Astropy.table.Table
         The modified table.
    
    """
    table = table.copy()
    if len(table) == 0:
        return table
    for col in table.colnames:
        #Remove empty columns
        try:
            if np.all(table[col].mask):
                print("Removing empty column: {}".format(col))
                table.remove_column(col)
                continue
        except AttributeError:
            print("{} is not a masked columns".format(col))
            
        #Get rid of column type object from VO queries
        if table[col].dtype == 'object':
            print("Converting column {} type from object to string".format(col) )
            table[col] = table[col].astype(str)
 
        #Get rid of unit '-' from some tables
        if table[col].unit == '-':
            print("Converting column {} unit from '-' to None".format(col) )
            table[col].unit = None   
            
        #replace masked floats with nans     
        if table[col].dtype == float:
            table[col].fill_value = np.nan
    
    table = table.filled()
            
    return table

In [12]:
v_flux = clean_table(v_flux)

In [13]:
import astropy.units as u

In [14]:
import astropy.units as u
for col in v_flux.colnames:
    if col.startswith('f'):
        v_flux[col] /= 1.E6
        v_flux[col].unit = u.Jansky


In [15]:
v_flux['ra'].convert_unit_to(u.rad)
v_flux['dec'].convert_unit_to(u.rad)
v_flux['ra'].name = 'v_ra'
v_flux['dec'].name = 'v_dec'

In [16]:
v_flux['f_vista_j'].name = 'j_flux'
v_flux['ferr_vista_j'].name = 'j_fluxErr'
v_flux['f_vista_h'].name = 'h_flux'
v_flux['ferr_vista_h'].name = 'h_fluxErr'
v_flux['f_vista_ks'].name = 'ks_flux'
v_flux['ferr_vista_ks'].name = 'ks_fluxErr'

In [17]:
import logging
LOGGER = logging.getLogger(__name__)
from astropy.coordinates import SkyCoord, Angle
import astropy.units as u
from collections import Counter
from astropy.table import Table, Column, hstack, vstack
def merge_catalogues(cat_1, cat_2, racol_2, decol_2, radius=0.4*u.arcsec):
    """Merge two catalogues
    This function merges the second catalogue into the first one using the
    given radius to associate identical sources.  This function takes care to
    associate only one source of one catalogue to the other.  The sources that
    may be associated to various counterparts in the other catalogue are
    flagged as “maybe spurious association” with a true value in the
    flag_merged column.  If this column is present in the first catalogue, it's
    content is “inherited” during the merge.
    Parameters
    ----------
    cat_1: astropy.table.Table
        The table containing the first catalogue.  This is the master catalogue
        used during the merge.  If it has a “flag_merged” column it's content
        will be re-used in the flagging of the spurious merges.  This catalogue
        must contain a ‘ra’ and a ‘dec’ columns with the position in decimal
        degrees.
    cat_2: astropy.table.Table
        The table containing the second catalogue.
    racol_2: string
        Name of the column in the second table containing the right ascension
        in decimal degrees.
    decol_2: string
        Name of the column in the second table containing the declination in
        decimal degrees.
    radius: astropy.units.quantity.Quantity
        The radius to associate identical sources in the two catalogues.
    Returns
    -------
    astropy.table.Table
        The merged catalogue.
    """
    cat_1 = cat_1.copy()
    cat_2 = cat_2.copy()
    cat_1['ra'].unit = u.rad
    cat_1['dec'].unit = u.rad
    coords_1 = SkyCoord(cat_1['ra'], cat_1['dec'])

    
    cat_2[racol_2].unit = u.rad
    cat_2[decol_2].unit = u.rad
    coords_2 = SkyCoord(cat_2[racol_2], cat_2[decol_2])

    # Search for sources in second catalogue matching the sources in the first
    # one.
    idx_2, idx_1, d2d, _ = coords_1.search_around_sky(coords_2, radius)

    # We want to flag the possible mis-associations, i.e. the sources in each
    # catalogue that are associated to several sources in the other one, but
    # also all the sources that are associated to a problematic source in the
    # other catalogue (e.g. if two sources in the first catalogue are
    # associated to the same source in the second catalogue, they must be
    # flagged as potentially problematic).
    #
    # Search for duplicate associations
    toflag_idx_1 = np.unique([item for item, count in Counter(idx_1).items()
                              if count > 1])
    toflag_idx_2 = np.unique([item for item, count in Counter(idx_2).items()
                              if count > 1])
    # Flagging the sources associated to duplicates
    dup_associated_in_idx1 = np.in1d(idx_2, toflag_idx_2)
    dup_associated_in_idx2 = np.in1d(idx_1, toflag_idx_1)
    toflag_idx_1 = np.unique(np.concatenate(
        (toflag_idx_1, idx_1[dup_associated_in_idx1])
    ))
    toflag_idx_2 = np.unique(np.concatenate(
        (toflag_idx_2, idx_2[dup_associated_in_idx2])
    ))

    # Adding the flags to the catalogue.  In the second catalogue, the column
    # is named "flag_merged_2" and will be combined to the flag_merged column
    # one the merge is done.
    try:
        cat_1["flag_merged"] |= np.in1d(np.arange(len(cat_1), dtype=int),
                                        toflag_idx_1)
    except KeyError:
        cat_1.add_column(Column(
            data=np.in1d(np.arange(len(cat_1), dtype=int), toflag_idx_1),
            name="flag_merged"
        ))
    try:
        cat_2["flag_merged_2"] |= np.in1d(np.arange(len(cat_2), dtype=int), toflag_idx_2)
    except KeyError:
        cat_2.add_column(Column(
            data=np.in1d(np.arange(len(cat_2), dtype=int), toflag_idx_2),
            name="flag_merged_2"
        ))


    # Now that we have flagged the maybe spurious associations, we want to
    # associate each source of each catalogue to at most one source in the
    # other one.

    # We sort the indices by the distance to take the nearest counterparts in
    # the following steps.
    sort_idx = np.argsort(d2d)
    idx_1 = idx_1[sort_idx]
    idx_2 = idx_2[sort_idx]

    # These array will contain the indexes of the matching sources in both
    # catalogues.
    match_idx_1 = np.array([], dtype=int)
    match_idx_2 = np.array([], dtype=int)

    while len(idx_1) > 0:

        both_first_idx = np.sort(np.intersect1d(
            np.unique(idx_1, return_index=True)[1],
            np.unique(idx_2, return_index=True)[1],
        ))

        new_match_idx_1 = idx_1[both_first_idx]
        new_match_idx_2 = idx_2[both_first_idx]

        match_idx_1 = np.concatenate((match_idx_1, new_match_idx_1))
        match_idx_2 = np.concatenate((match_idx_2, new_match_idx_2))

        # We remove the matching sources in both catalogues.
        to_remove = (np.in1d(idx_1, new_match_idx_1) |
                     np.in1d(idx_2, new_match_idx_2))
        idx_1 = idx_1[~to_remove]
        idx_2 = idx_2[~to_remove]

    # Indices of un-associated object in both catalogues.
    unmatched_idx_1 = np.delete(np.arange(len(cat_1), dtype=int),match_idx_1)
    unmatched_idx_2 = np.delete(np.arange(len(cat_2), dtype=int),match_idx_2)

    # Sources only in cat_1
    only_in_cat_1 = cat_1[unmatched_idx_1]

    # Sources only in cat_2
    only_in_cat_2 = cat_2[unmatched_idx_2]
    # We are using the ra and dec columns from cat_2 for the position.
    only_in_cat_2[racol_2].name = "ra"
    only_in_cat_2[decol_2].name = "dec"

    # Merged table of sources in both catalogues.
    both_in_cat_1_and_cat_2 = hstack([cat_1[match_idx_1], cat_2[match_idx_2]])
    # We don't need the positions from the second catalogue anymore.
    both_in_cat_1_and_cat_2.remove_columns([racol_2, decol_2])

    # Logging the number of rows
    LOGGER.info("There are %s sources only in the first catalogue",
                len(only_in_cat_1))
    LOGGER.info("There are %s sources only in the second catalogue",
                len(only_in_cat_2))
    LOGGER.info("There are %s sources in both catalogues",
                len(both_in_cat_1_and_cat_2))

    merged_catalogue = vstack([only_in_cat_1, both_in_cat_1_and_cat_2,
                               only_in_cat_2])

    # When vertically stacking the catalogues, some values in the flag columns
    # are masked because they did not exist in the catalogue some row originate
    # from. We must set them to the appropriate value.
    for colname in merged_catalogue.colnames:
        if 'flag' in colname:
            merged_catalogue[colname][merged_catalogue[colname].mask] = False

    # We combined the flag_merged flags
    merged_catalogue['flag_merged'] |= merged_catalogue['flag_merged_2']
    merged_catalogue.remove_column('flag_merged_2')
    merged_catalogue.remove_column('flag_merged')
    return merged_catalogue

In [18]:
r_cat = Table.read("../dmu0/dmu0_PanStarrs/data/ps1_pv3_3pi_20170110_GmagLT19/133200.fits")

In [19]:
r_cat['flags'].format

In [23]:
fits.open('./data/refcats/{}.fits'.format(c))[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  205 / length of dimension 1                          
NAXIS2  =                  119 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   29 / number of table fields                         
TTYPE1  = 'flags   '                                                            
TFORM1  = '1X      '                                                            
TTYPE2  = 'id      '                                                            
TFORM2  = '1K      '                                                            
TTYPE3  = 'coord_ra'        

In [26]:
fits.open('../dmu0/dmu0_PanSTARRS/data/ps1_pv3_3pi_20170110_GmagLT19/133200.fits')[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional binary table                     
NAXIS1  =                  157 / width of table in bytes                        
NAXIS2  =                  250 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group (required keyword)              
TFIELDS =                   23 / number of fields in each row                   
TTYPE1  = 'flags   '           / bits for all Flag fields; see also TFLAGn      
TFORM1  = '1X      '           / format of field                                
FLAGCOL =                    1 / Column number for the bitflags.                
TTYPE2  = 'id      '           / unique ID                                      
TFORM2  = '1K      '        

In [33]:
col_dict={
'flags':'1X',
 'id':'1K',
 'coord_ra':'1D',
 'coord_dec':'1D',
 'g_flux':'1D',
 'r_flux':'1D',
 'i_flux':'1D',
 'z_flux':'1D',
 'y_flux':'1D',
 'g_fluxErr':'1D',
 'r_fluxErr':'1D',
 'i_fluxErr':'1D',
 'z_fluxErr':'1D',
 'y_fluxErr':'1D',
 'coord_raErr':'1E',
 'coord_decErr':'1E',
 'epoch':'1D',
 'pm_ra':'1D',
 'pm_dec':'1D',
 'pm_raErr':'1E',
 'pm_decErr':'1E',
 'parent':'1K',
 'footprint':'1J',
 'j_flux':'1D',
 'j_fluxErr':'1D',
 'h_flux':'1D',
 'h_fluxErr':'1D',
 'ks_flux':'1D',
 'ks_fluxErr':'1D'}

cls_dict={
'flags':'',
 'id':'',
 'coord_ra':'Angle',
 'coord_dec':'Angle',
 'g_flux':'Scalar',
 'r_flux':'Scalar',
 'i_flux':'Scalar',
 'z_flux':'Scalar',
 'y_flux':'Scalar',
 'g_fluxErr':'Scalar',
 'r_fluxErr':'Scalar',
 'i_fluxErr':'Scalar',
 'z_fluxErr':'Scalar',
 'y_fluxErr':'Scalar',
 'coord_raErr':'Scalar',
 'coord_decErr':'Scalar',
 'epoch':'Scalar',
 'pm_ra':'Angle',
 'pm_dec':'Angle',
 'pm_raErr':'Scalar',
 'pm_decErr':'Scalar',
 'parent':'Scalar',
 'footprint':'Scalar',
 'j_flux':'Scalar',
 'j_fluxErr':'Scalar',
 'h_flux':'Scalar',
 'h_fluxErr':'Scalar',
 'ks_flux':'Scalar',
 'ks_fluxErr':'Scalar'}

unit_dict={
'flags':'',
 'id':'',
 'coord_ra':'rad',
 'coord_dec':'rad',
 'g_flux':'Jy',
 'r_flux':'Jy',
 'i_flux':'Jy',
 'z_flux':'Jy',
 'y_flux':'Jy',
 'g_fluxErr':'Jy',
 'r_fluxErr':'Jy',
 'i_fluxErr':'Jy',
 'z_fluxErr':'Jy',
 'y_fluxErr':'Jy',
 'coord_raErr':'rad',
 'coord_decErr':'rad',
 'epoch':'day',
 'pm_ra':'rad',
 'pm_dec':'rad',
 'pm_raErr':'rad/year',
 'pm_decErr':'rad/year',
 'parent':'',
 'footprint':'',
 'j_flux':'Jy',
 'j_fluxErr':'Jy',
 'h_flux':'Jy',
 'h_fluxErr':'Jy',
 'ks_flux':'Jy',
 'ks_fluxErr':'Jy'}

docs_dict={
'flags':'',
 'id':'',
 'coord_ra':'',
 'coord_dec':'',
 'g_flux':'',
 'r_flux':'',
 'i_flux':'',
 'z_flux':'',
 'y_flux':'',
 'g_fluxErr':'',
 'r_fluxErr':'',
 'i_fluxErr':'',
 'z_fluxErr':'',
 'y_fluxErr':'',
 'coord_raErr':'',
 'coord_decErr':'',
 'epoch':'',
 'pm_ra':'',
 'pm_dec':'',
 'pm_raErr':'',
 'pm_decErr':'',
 'parent':'',
 'footprint':'',
 'j_flux':'',
 'j_fluxErr':'',
 'h_flux':'',
 'h_fluxErr':'',
 'ks_flux':'',
 'ks_fluxErr':''}

In [ ]:
for c in ps_refcats:
    r_cat = Table.read("../dmu0/dmu0_PanStarrs/data/ps1_pv3_3pi_20170110_GmagLT19/{}.fits".format(c))
    r_cat['coord_ra'].name = 'ra'
    r_cat['coord_dec'].name = 'dec'
    merge = merge_catalogues(r_cat, v_flux, 'v_ra', 'v_dec', radius=0.4*u.arcsec)
    has_both  = (merge['ks_flux']>0.) & (merge['g_flux'] >0.)
    merge['ra'].name = 'coord_ra'
    merge['dec'].name = 'coord_dec'
    merge['flags'].format = '1X'
    merge['epoch'] = merge['epoch'].astype('int32')
    #merge[has_both].write('./data/refcats/{}.fits'.format(c), overwrite=True)
    merge = merge[has_both]
    f = fits.BinTableHDU.from_columns([
    fits.Column(
        name=merge[col].name,
        array=merge[col].data, 
        format=col_dict[col]) 
    for col in merge.colnames
    ])
    f.header['AFW_TYPE']= 'SIMPLE'
    f.header['HIERARCH AFW_TABLE_VERSION'] = 3 
    f.header['FLAGCOL'] = 1
    f.header['TFLAG1'] = 'pm_flag'
    for key in f.header:
        if key.startswith('TFORM'):
            num = key[5:]
            if not cls_dict[f.header['TTYPE'+str(num)]]=='':
                f.header['TCCLS'+str(num)] = cls_dict[f.header['TTYPE'+str(num)]]
                
            if not unit_dict[f.header['TTYPE'+str(num)]]=='':
                f.header['TUNIT'+str(num)] = unit_dict[f.header['TTYPE'+str(num)]]
            f.header['TDOC'+str(num)] = docs_dict[f.header['TTYPE'+str(num)]]
    f.writeto('./data/refcats/{}.fits'.format(c), overwrite=True)

In [ ]:
'./data/refcats/{}.fits'.format(c)

In [ ]:
f.data

In [ ]:
fits.open('./data/refcats/{}.fits'.format(c), overwrite=True)[0].header

In [ ]:
fits.open('./data/refcats/{}.fits'.format(c), overwrite=True)[1].header

In [ ]:
#merge.remove_columns('flags')
f = fits.BinTableHDU.from_columns([
    fits.Column(
        name=merge[col].name,
        array=merge[col].data[:0], 
        format=col_dict[col]) 
    for col in merge.colnames
])
f.header['AFW_TYPE']= 'SOURCE'
f.header['HIERARCH AFW_TABLE_VERSION'] = 1 
f.header['FLAGCOL'] = 1
for key in f.header:
    if key.startswith('TFORM'):
        num = key[5:]
        if not cls_dict[f.header['TTYPE'+str(num)]]=='':
            f.header['TCCLS'+str(num)] = cls_dict[f.header['TTYPE'+str(num)]]
                
        if not unit_dict[f.header['TTYPE'+str(num)]]=='':
            f.header['TUNIT'+str(num)] = unit_dict[f.header['TTYPE'+str(num)]]
        f.header['TDOC'+str(num)] = docs_dict[f.header['TTYPE'+str(num)]]
hdu2=fits.open('../dmu0/dmu0_PanSTARRS/data/ps1_pv3_3pi_20170110_GmagLT19/master_schema.fits')[2]
hdu = fits.HDUList([fits.PrimaryHDU(), f,hdu2])
hdu.writeto('./data/refcats/master_schema.fits', overwrite=True)
f.writeto('./data/refcats/master_schema.fits', overwrite=True)

In [ ]:
f.header

In [ ]:
fn=fits.open('./data/refcats/master_schema.fits')

In [ ]:
fn[2].header

In [ ]:
hdu=fits.open('../dmu0/dmu0_PanSTARRS/data/ps1_pv3_3pi_20170110_GmagLT19/master_schema.fits')

In [ ]:
hdu0=hdu[0]
hdu1=hdu[1]
hdu2=hdu[2]

In [ ]:
hdu2.header

In [ ]:
sc = afwTable.SourceCatalog

In [ ]:
sc.schema

In [ ]:
afwTable.SimpleCatalog.readFits(EX_CAT)

In [ ]:
T = '../dmu1/data/refcats/134080.fits'
t = afwTable.SimpleCatalog.readFits(T)
t

In [ ]:
t.writeFits('./data/refcats/134080.fits')

In [ ]:
sc.

In [ ]:
io = afwTable.

In [ ]:
TS = '../dmu1/data/refcats/master_schema.fits'
ts = afwTable.SimpleCatalog.readFits(TS)
ts

In [ ]:
afwTable.Schema.

In [ ]:
afwTable.SourceCatalog().makeMinimalSchema()